# Comparison between "Comprehensive upper-air observation network from 1905 to present" and "Insitu IGRA radiosoundings baseline network"

**Contains modified Copernicus Climate Change Service Information 2020**
under [License](https://apps.ecmwf.int/datasets/licences/copernicus/)

*Copernicus Climate Change Service (C3S) - Upper Air Data Service (2020)*

The purpose of this IPython Notebook is to find differences in both mentioned data sets and to compare them. 

    Author: U. Voggenberger
    Date: 02.2021
    Contact: ulrich.voggenberger [at] univie.ac.at
    License: C3S, 2021

In [11]:
import pandas
import numpy as np
import sys, zipfile, os, time
import matplotlib.pyplot as plt
import glob
import datetime
import urllib3
import cdsapi
import xarray
import shutil
import warnings
import pickle
warnings.filterwarnings('ignore')

sys.path.append(os.getcwd()+'/../cds-backend/code/')
import cds_eua3 as eua


In [49]:
def request(rqdict, source, remove_file=True):
    
    t0 = time.time()

    c = cdsapi.Client()
    r = c.retrieve(
        source,rqdict)
    if True:
        r.download(target='download.zip')
        assert os.stat('download.zip').st_size == r.content_length, "Downloaded file is incomplete"
    z = zipfile.ZipFile('download.zip')
    z.extractall(path='./download/')
    z.close()
    print('Request took: ' + str(time.time() - t0) + ' seconds')
    
    files = glob.glob('./download/*.nc')
    
    if files[0].split('/')[-1].startswith('IGRA'):
        ds = xarray.open_dataset(files[0])
        ds = ds.drop(['report_id'])
        data = ds.to_dataframe()
        for i in files[1:]:
            ds = xarray.open_dataset(i)            
            data = data.append(ds.to_dataframe())

    else:
        data=eua.CDMDataset(files[0]).to_dataframe()
        for i in files[1:]:
            da = eua.CDMDataset(i).to_dataframe()
            data = data.append(da)
            
    os.remove('download.zip')
    if remove_file:
        try:
            shutil.rmtree('./download/')
        except:
            print('Error while deleting directory')

    return data

---

In [50]:
cuon = 'insitu-comprehensive-upper-air-observation-network'
igra = 'insitu-observations-igra-baseline-network'

# short period - single station

In [67]:
t = '1990-01-01/1990-01-02'
li =  ['AUM00011035']
lc = ['11035']

### CUON

In [68]:
request({
    'variable':'temperature',
    'period':t,
    'statid':lc,
},cuon)

2021-02-10 10:30:54,158 INFO Welcome to the CDS
2021-02-10 10:30:54,160 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-comprehensive-upper-air-observation-network
2021-02-10 10:30:54,387 INFO Request is completed
2021-02-10 10:30:54,389 INFO Downloading http://136.156.132.176/cache-compute-0002/cache/data1/adaptor.comprehensive_upper_air.retrieve-1612949398.6048949-7689-4-368dafef-a83d-4962-bfc9-fe6eb3fe3101.zip to download.zip (37.8K)
2021-02-10 10:30:54,476 INFO Download rate 437.2K/s
2021-02-10 10:30:54,495 INFO Reading Profile on slice(None, None, None)


Request took: 0.35447120666503906 seconds


,lat,lon,obs,plev,report_id,station_id,ta,time,trajectory_label
0,48.25,16.370001,0,1230.0,10000016000,WIEN/HOHE WARTE (11035-0),202.699997,1990-01-01 00:00:00,10000016001
1,48.25,16.370001,0,1490.0,10000016000,WIEN/HOHE WARTE (11035-0),202.899994,1990-01-01 00:00:00,10000016001
2,48.25,16.370001,0,2000.0,10000016000,WIEN/HOHE WARTE (11035-0),207.500000,1990-01-01 00:00:00,10000016001
3,48.25,16.370001,0,2170.0,10000016000,WIEN/HOHE WARTE (11035-0),209.300003,1990-01-01 00:00:00,10000016001
4,48.25,16.370001,0,3000.0,10000016000,WIEN/HOHE WARTE (11035-0),211.100006,1990-01-01 00:00:00,10000016001
...,...,...,...,...,...,...,...,...,...
129,48.25,16.370001,0,85600.0,10000016006,WIEN/HOHE WARTE (11035-0),267.500000,1990-01-02 12:00:00,10000016007
130,48.25,16.370001,0,87400.0,10000016006,WIEN/HOHE WARTE (11035-0),266.700012,1990-01-02 12:00:00,10000016007
131,48.25,16.370001,0,91600.0,10000016006,WIEN/HOHE WARTE (11035-0),268.100006,1990-01-02 12:00:00,10000016007
132,48.25,16.370001,0,92400.0,10000016006,WIEN/HOHE WARTE (11035-0),267.899994,1990-01-02 12:00:00,10000016007


### IGRA

In [69]:
request({
    'source': 'IGRA',
    'variable': ['air_temperature'],
    'period':t,
    'station_name':li,
    }, igra)

2021-02-10 10:30:58,300 INFO Welcome to the CDS
2021-02-10 10:30:58,301 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-observations-igra-baseline-network
2021-02-10 10:30:58,353 INFO Request is completed
2021-02-10 10:30:58,354 INFO Downloading http://136.156.132.204/cache-compute-0001/cache/data2/adaptor.insitu_reference.retrieve_test-1612949412.1758745-20651-5-c264dd21-854d-48ba-bdbc-705202a7d7d3.zip to download.zip (2.3K)
2021-02-10 10:30:58,393 INFO Download rate 60.4K/s


Request took: 0.13030552864074707 seconds


,station_name,report_timestamp,longitude,latitude,air_pressure,air_temperature
index,,,,,,
0,AUM00011035,1990-01-02,16.3564,48.2486,99800,271.850006
1,AUM00011035,1990-01-02,16.3564,48.2486,98500,NaN
2,AUM00011035,1990-01-02,16.3564,48.2486,96100,268.850006
3,AUM00011035,1990-01-02,16.3564,48.2486,96000,NaN
4,AUM00011035,1990-01-02,16.3564,48.2486,93700,NaN
5,AUM00011035,1990-01-02,16.3564,48.2486,92400,NaN
6,AUM00011035,1990-01-02,16.3564,48.2486,89800,265.450012
7,AUM00011035,1990-01-02,16.3564,48.2486,88600,268.450012
8,AUM00011035,1990-01-02,16.3564,48.2486,87500,269.649994


# long period - single station

In [80]:
t = '1990-01-01/2010-01-02'
li =  ['AUM00011035']
lc = ['11035']

### CUON

In [81]:
request({
    'variable':'temperature',
    'period':t,
    'statid':lc,
},cuon)

2021-02-10 11:09:39,114 INFO Welcome to the CDS
2021-02-10 11:09:39,115 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-comprehensive-upper-air-observation-network
2021-02-10 11:09:39,277 INFO Request is queued
2021-02-10 11:09:40,315 INFO Request is running
2021-02-10 11:12:30,851 INFO Request is completed
2021-02-10 11:12:30,853 INFO Downloading http://136.156.132.176/cache-compute-0000/cache/data1/adaptor.comprehensive_upper_air.retrieve-1612951907.3832922-17792-13-03b6bfbf-0f47-4de9-b126-deb392889bf4.zip to download.zip (27.7M)
2021-02-10 11:12:31,687 INFO Download rate 33.2M/s  
2021-02-10 11:12:31,740 INFO Reading Profile on slice(None, None, None)


Request took: 172.73495435714722 seconds


,lat,lon,obs,plev,report_id,station_id,ta,time,trajectory_label
0,48.25,16.370001,0,1230.0,10000016000,WIEN/HOHE WARTE (11035-0),202.699997,1990-01-01 00:00:00,10000016001
1,48.25,16.370001,0,1490.0,10000016000,WIEN/HOHE WARTE (11035-0),202.899994,1990-01-01 00:00:00,10000016001
2,48.25,16.370001,0,2000.0,10000016000,WIEN/HOHE WARTE (11035-0),207.500000,1990-01-01 00:00:00,10000016001
3,48.25,16.370001,0,2170.0,10000016000,WIEN/HOHE WARTE (11035-0),209.300003,1990-01-01 00:00:00,10000016001
4,48.25,16.370001,0,3000.0,10000016000,WIEN/HOHE WARTE (11035-0),211.100006,1990-01-01 00:00:00,10000016001
...,...,...,...,...,...,...,...,...,...
760856,48.25,16.370001,0,82700.0,10000045414,WIEN/HOHE WARTE (11035-0),263.100006,2010-01-02 12:00:00,10000045415
760857,48.25,16.370001,0,85000.0,10000045414,WIEN/HOHE WARTE (11035-0),264.899994,2010-01-02 12:00:00,10000045415
760858,48.25,16.370001,0,86500.0,10000045414,WIEN/HOHE WARTE (11035-0),266.100006,2010-01-02 12:00:00,10000045415
760859,48.25,16.370001,0,92500.0,10000045414,WIEN/HOHE WARTE (11035-0),270.700012,2010-01-02 12:00:00,10000045415


### IGRA

In [82]:
request({
    'source': 'IGRA',
    'variable': ['air_temperature'],
    'period':t,
    'station_name':li,
    }, igra)

2021-02-10 11:12:33,677 INFO Welcome to the CDS
2021-02-10 11:12:33,678 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-observations-igra-baseline-network
2021-02-10 11:12:33,818 INFO Request is queued
2021-02-10 11:12:34,857 INFO Request is running
2021-02-10 11:16:52,151 INFO Request is completed
2021-02-10 11:16:52,152 INFO Downloading http://136.156.132.176/cache-compute-0002/cache/data1/adaptor.insitu_reference.retrieve_test-1612952188.996873-14762-10-ca8b9086-484c-485c-9690-98cfcd7c6a4e.zip to download.zip (8.8M)
2021-02-10 11:16:52,557 INFO Download rate 21.7M/s  


Request took: 259.13984203338623 seconds


,station_name,report_timestamp,longitude,latitude,air_pressure,air_temperature
index,,,,,,
0,AUM00011035,1990-01-02,16.3564,48.2486,99800,271.850006
1,AUM00011035,1990-01-02,16.3564,48.2486,98500,NaN
2,AUM00011035,1990-01-02,16.3564,48.2486,96100,268.850006
3,AUM00011035,1990-01-02,16.3564,48.2486,96000,NaN
4,AUM00011035,1990-01-02,16.3564,48.2486,93700,NaN
5,AUM00011035,1990-01-02,16.3564,48.2486,92400,NaN
6,AUM00011035,1990-01-02,16.3564,48.2486,89800,265.450012
7,AUM00011035,1990-01-02,16.3564,48.2486,88600,268.450012
8,AUM00011035,1990-01-02,16.3564,48.2486,87500,269.649994


# short period - multiple stations

In [73]:
t = '1990-01-01/1990-02-01'
li =  ['AUM00011035', 'UKM00003693', 'GMM00010238', 'ITM00016245']
lc = ['11035', '03693', '10238', '16245']

### CUON

In [74]:
request({
    'variable':'temperature',
    'period':t,
    'statid':lc,
},cuon)

2021-02-10 10:39:01,679 INFO Welcome to the CDS
2021-02-10 10:39:01,680 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-comprehensive-upper-air-observation-network
2021-02-10 10:39:01,751 INFO Request is queued
2021-02-10 10:39:02,788 INFO Request is running
2021-02-10 10:39:51,348 INFO Request is completed
2021-02-10 10:39:51,349 INFO Downloading http://136.156.132.204/cache-compute-0001/cache/data2/adaptor.comprehensive_upper_air.retrieve-1612949974.7329144-8122-1-7467d4e5-3742-432e-a739-391260254f5b.zip to download.zip (438.5K)
2021-02-10 10:39:51,563 INFO Download rate 2M/s   
2021-02-10 10:39:51,585 INFO Reading Profile on slice(None, None, None)
2021-02-10 10:39:51,599 INFO Reading Profile on slice(None, None, None)
2021-02-10 10:39:51,614 INFO Reading Profile on slice(None, None, None)
2021-02-10 10:39:51,635 INFO Reading Profile on slice(None, None, None)


Request took: 49.92758560180664 seconds


,lat,lon,obs,plev,report_id,station_id,ta,time,trajectory_label
0,52.82,9.930000,0,15000.0,10000004836,BERGEN,219.300003,1990-01-02 05:00:00,10000004836
1,52.82,9.930000,0,20000.0,10000004836,BERGEN,218.899994,1990-01-02 05:00:00,10000004836
2,52.82,9.930000,0,25000.0,10000004836,BERGEN,214.300003,1990-01-02 05:00:00,10000004836
3,52.82,9.930000,0,30000.0,10000004836,BERGEN,221.100006,1990-01-02 05:00:00,10000004836
4,52.82,9.930000,0,40000.0,10000004836,BERGEN,236.899994,1990-01-02 05:00:00,10000004836
...,...,...,...,...,...,...,...,...,...
2391,48.25,16.370001,0,88100.0,10000016125,WIEN/HOHE WARTE (11035-0),277.399994,1990-02-01 12:00:00,10000016126
2392,48.25,16.370001,0,90500.0,10000016125,WIEN/HOHE WARTE (11035-0),275.399994,1990-02-01 12:00:00,10000016126
2393,48.25,16.370001,0,92300.0,10000016125,WIEN/HOHE WARTE (11035-0),273.600006,1990-02-01 12:00:00,10000016126
2394,48.25,16.370001,0,92500.0,10000016125,WIEN/HOHE WARTE (11035-0),273.600006,1990-02-01 12:00:00,10000016126


### IGRA

In [ ]:
display(    
    request({
        'source': 'IGRA',
        'variable': ['air_temperature'],
        'period':t,
        'station_name':li,
    }, igra)


In [79]:
for i in li:
    print(i)
    display(    
        request({
            'source': 'IGRA',
            'variable': ['air_temperature'],
            'period':t,
            'station_name':[i],
        }, igra)
    )

2021-02-10 10:59:48,991 INFO Welcome to the CDS
2021-02-10 10:59:48,992 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-observations-igra-baseline-network
2021-02-10 10:59:49,127 INFO Request is queued


AUM00011035


2021-02-10 10:59:50,165 INFO Request is completed
2021-02-10 10:59:50,166 INFO Downloading http://136.156.132.204/cache-compute-0001/cache/data2/adaptor.insitu_reference.retrieve_test-1612951189.481721-8468-6-b44a4587-958f-4716-9435-d719db6ddf20.zip to download.zip (26.8K)
2021-02-10 10:59:50,239 INFO Download rate 373.5K/s


Request took: 1.2876148223876953 seconds


,station_name,report_timestamp,longitude,latitude,air_pressure,air_temperature
index,,,,,,
0,AUM00011035,1990-01-02,16.3564,48.2486,99800,271.850006
1,AUM00011035,1990-01-02,16.3564,48.2486,98500,NaN
2,AUM00011035,1990-01-02,16.3564,48.2486,96100,268.850006
3,AUM00011035,1990-01-02,16.3564,48.2486,96000,NaN
4,AUM00011035,1990-01-02,16.3564,48.2486,93700,NaN
5,AUM00011035,1990-01-02,16.3564,48.2486,92400,NaN
6,AUM00011035,1990-01-02,16.3564,48.2486,89800,265.450012
7,AUM00011035,1990-01-02,16.3564,48.2486,88600,268.450012
8,AUM00011035,1990-01-02,16.3564,48.2486,87500,269.649994


...

2021-02-10 10:59:52,850 INFO Welcome to the CDS
2021-02-10 10:59:52,851 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-observations-igra-baseline-network
2021-02-10 10:59:52,892 INFO Request is queued


ITM00016245


2021-02-10 10:59:53,928 INFO Request is completed
2021-02-10 10:59:53,930 INFO Downloading http://136.156.132.176/cache-compute-0000/cache/data1/adaptor.insitu_reference.retrieve_test-1612951193.2677338-32474-7-f555e11a-19d1-426c-892f-78e539721767.zip to download.zip (49.5K)
2021-02-10 10:59:54,017 INFO Download rate 572.9K/s


Request took: 1.2039248943328857 seconds


,station_name,report_timestamp,longitude,latitude,air_pressure,air_temperature
index,,,,,,
0,AUM00011035,1990-01-02,16.3564,48.2486,99800,271.850006
1,AUM00011035,1990-01-02,16.3564,48.2486,98500,NaN
2,AUM00011035,1990-01-02,16.3564,48.2486,96100,268.850006
3,AUM00011035,1990-01-02,16.3564,48.2486,96000,NaN
4,AUM00011035,1990-01-02,16.3564,48.2486,93700,NaN
5,AUM00011035,1990-01-02,16.3564,48.2486,92400,NaN
6,AUM00011035,1990-01-02,16.3564,48.2486,89800,265.450012
7,AUM00011035,1990-01-02,16.3564,48.2486,88600,268.450012
8,AUM00011035,1990-01-02,16.3564,48.2486,87500,269.649994


# long period - multiple stations

In [ ]:
t = '1990-01-01/1990-02-01'
li =  ['AUM00011035', 'UKM00003693', 'GMM00010238', 'ITM00016245']
lc = ['11035', '03693', '10238', '16245']

### CUON

In [ ]:
request({
    'variable':'temperature',
    'period':t,
    'statid':lc,
},cuon)

### IGRA 

In [ ]:
request({
    'source': 'IGRA',
    'variable': ['air_temperature'],
    'period':t,
    'station_name':li,
    }, igra)